In [2]:
import pandas as pd
from fuzzywuzzy import process

Este notebook es para pegarle la informacion del snies a los resultados del valor agregado enviados por el icfes. El objetivo es armar el identificador icine, con el fin de comparar los resultados del icfes 2021-2022, con los resultados propios 2022-2023. El resultado se le envió a Julian el 16 de julio para la presentación con victor el 17 de julio. 

Esto lo haremos:

1. pegandole la llave de la bace cine a la base del icves,módulo diferencias de sintaxis
2. Una vez la base del icfes tiene la llave del cine, pegarle la info del cine.
3. construir la columna icine (icine especifico)

In [3]:
cols_base_cine = [
    'codigo_institucion',
    'codigo_institucion_padre',
    'nombre_institucion',
    'estado_institucion',
    'caracter_academico',
    'codigo_snies_del_programa',
    'nombre_del_programa',
    'estado_programa',
    'cine_f_2013_ac_campo_especific',
    'nucleo_basico_del_conocimiento',
    'nivel_academico',
    'municipio_oferta_programa',
    'id_cine_campo_especifico',
    'codigo_del_municipio_programa',
    'codigo_de_la_institucion',
    'institucion_de_educacion_superior_ies',
    'ies_acreditada',
    'caracter_ies',
    'municipio'
]

In [4]:
cols_icfes = [
    'inst_cod_institucion', 
    'NBC2',
    'INBC',
    'va_lectura_critica',
    'va_razona_cuantitat',
]

In [5]:
cine_nbc_programas_file = "base_cine_bogota_programas_vigencia_2021_2022_15072025.xlsx"
cine_nbc_programas_path = "../../data/SNIES_CINE_cleaned/"

va_icfes_path = "../../data/Resultados_ICFES_VA/"
va_icfes_file = "VA_INBCs_Bogota.xlsx"

In [6]:
cine_nbc_programas = pd.read_excel(cine_nbc_programas_path + cine_nbc_programas_file, usecols = cols_base_cine)
va_icfes = pd.read_excel(va_icfes_path + va_icfes_file, usecols = cols_icfes)

In [7]:
cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'] = (
    cine_nbc_programas['nucleo_basico_del_conocimiento'].str.upper()
)

In [8]:
print(len(cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'].sort_values().unique()))
print(len(va_icfes["NBC2"].sort_values().unique()))

55
48


In [9]:
# Listas únicas
nbc_1 = cine_nbc_programas['nucleo_basico_del_conocimiento_mayus'].dropna().unique()
nbc_2 = va_icfes["NBC2"].dropna().unique()

# Empatar cada item de nbc_1 con el más similar en nbc_2
matches = []

for item in nbc_1:
    match, score = process.extractOne(item, nbc_2)
    matches.append((item, match, score))

# Convertir a DataFrame para ver el resultado
fuzzy_matches = pd.DataFrame(matches, columns=["NBC_cine", "NBC_icfes", "similarity_score"])

# Filtrar los que tengan score alto (>90 por ejemplo)
fuzzy_matches_high = fuzzy_matches[fuzzy_matches["similarity_score"] >= 90]
fuzzy_matches_high_renamed = fuzzy_matches_high.rename(columns={"NBC_icfes": "NBC2"})

va_icfes_merged = va_icfes.merge(
    fuzzy_matches_high_renamed[["NBC2", "NBC_cine"]],
    on="NBC2",
    how="left"
)

Mirar cuales nbc quedaron por fuera

In [10]:
#✅ Paso 1: Asegúrate de eliminar NaN y duplicados
nbc2_unique = va_icfes["NBC2"].dropna().unique()
matched_nbc2 = fuzzy_matches_high["NBC_icfes"].dropna().unique()

#✅ Paso 2: Verifica si todos están presentes
all_matched = set(nbc2_unique).issubset(set(matched_nbc2))
print("¿Todos los valores de NBC2 están en fuzzy_matches_high?", all_matched)
#Paso 3 (opcional): Ver qué valores faltan
missing_nbc2 = set(nbc2_unique) - set(matched_nbc2)
print("Valores de NBC2 que no fueron emparejados:")
for val in sorted(missing_nbc2):
    print("-", val)

¿Todos los valores de NBC2 están en fuzzy_matches_high? False
Valores de NBC2 que no fueron emparejados:
- BACTERIOLOGIA


Enriquecemos el dataframe del icfes con la información del snies

In [12]:
va_icfes_enriquecido = va_icfes_merged.merge(
    cine_nbc_programas,
    left_on=["inst_cod_institucion","NBC_cine"],
    right_on=["codigo_institucion","nucleo_basico_del_conocimiento_mayus"],
    how="left"
)

va_icfes_enriquecido["icine_especifico"] = (
    va_icfes_enriquecido["inst_cod_institucion"].astype(str) + "_" +
    va_icfes_enriquecido["cine_f_2013_ac_campo_especific"].astype(str)
)

va_icfes_enriquecido['periodo'] = "2021-2022"
va_icfes_enriquecido.codigo_snies_del_programa = va_icfes_enriquecido.codigo_snies_del_programa.astype("Int64")

In [13]:
campos_exportar = [
    'INBC',
    'icine_especifico',
    'municipio_oferta_programa',
    'va_lectura_critica',
    'va_razona_cuantitat',
    'codigo_institucion',
    'nombre_institucion',
    'codigo_snies_del_programa',
    'nombre_del_programa',
    "periodo"
]


In [14]:
va_icfes_enriquecido[campos_exportar].reset_index(drop=True).to_excel("valor_agregado_xprograma_2021_2022.xlsx",index=False)

mirar cuantos programas aperecerían tendrían más de 2 icines

In [15]:
# Filtrar los duplicados excluyendo NaNs
va_icfes_enriquecido[
    va_icfes_enriquecido['codigo_snies_del_programa'].notna() & 
    va_icfes_enriquecido.duplicated(subset='codigo_snies_del_programa', keep=False)
]

,inst_cod_institucion,NBC2,INBC,va_lectura_critica,va_razona_cuantitat,NBC_cine,codigo_institucion,codigo_institucion_padre,nombre_institucion,estado_institucion,...,id_cine_campo_especifico,codigo_del_municipio_programa,codigo_de_la_institucion,institucion_de_educacion_superior_ies,ies_acreditada,caracter_ies,municipio,nucleo_basico_del_conocimiento_mayus,icine_especifico,periodo
